In [1]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import openai
import getpass
import logging

In [2]:
openai.api_base = "http://172.16.3.115:2024/v1"
openai.api_key = "sk-CFvfPTt9gypWFMsY0mWSd29mysqqqqqqqqqqqq111"
openai_model_name="qwen2.5-7b"

prompt_template = PromptTemplate(
        input_variables=["ticket_content", "executed_commands"],
        template="""\
你是一名专业的IT运维审计助手，负责分析工单执行情况。以下是输入内容：
1. 原始工单内容：
{ticket_content}

2. 实际执行的命令：
{executed_commands}

请完成以下任务：
1. 对比工单内容和实际执行命令，分析命令是否完全一致。
2. 如果存在不一致，请明确指出工单中的命令与实际执行命令的差异，以及可能的原因。
3. 检查实际执行的命令是否完全按照工单的步骤顺序执行。
4. 检查是否存在额外的、不合规的命令操作，并详细说明原因及风险。
5. 最后，生成一份总结报告，内容包括：
   - 执行合规性分析（符合/不符合）。
   - 不一致命令及原因分析。
   - 不合规命令的风险说明。
   - 提供明确的审批是否通过的结论。
"""
    )

openai_llm = ChatOpenAI(
        model=openai_model_name,
        openai_api_base=openai.api_base,
        openai_api_key=openai.api_key,
        temperature=0.7,
        max_tokens=1500
    )

ollama_base = "http://172.16.3.115:21434"
ollama_model_name="qwen2.5"

ollama_llm = ChatOllama(
        model=ollama_model_name,
        temperature=0.7,
        base_url=ollama_base
    )

openai_chain = LLMChain(llm=openai_llm, prompt=prompt_template)
ollama_chain = LLMChain(llm=ollama_llm, prompt=prompt_template)

/tmp/ipykernel_1568848/3004803029.py:45: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  openai_chain = LLMChain(llm=openai_llm, prompt=prompt_template)


In [3]:
ticket_content = """\
升级背景：系统需要从 v1.0.0 升级到 v2.0.0，以修复安全漏洞并优化性能。

升级步骤：
1. 停止相关服务：systemctl stop webserver
2. 备份数据目录：cp -r /var/www/html /backup/html_backup
3. 安装升级包：dpkg -i /updates/webserver_v2.0.0.deb
4. 检查版本号：webserver --version
5. 启动服务：systemctl start webserver
6. 检查服务状态：systemctl status webserver
"""
executed_commands = """
[
    {
        "序号": 1,
        "执行时间": "2024-11-21 10:00:00",
        "是否允许执行": true,
        "命令": "systemctl stop webserver"
    },
    {
        "序号": 2,
        "执行时间": "2024-11-21 10:05:00",
        "是否允许执行": true,
        "命令": "cp -r /var/www/html /backup/html_backup"
    },
    {
        "序号": 3,
        "执行时间": "2024-11-21 10:10:00",
        "是否允许执行": true,
        "命令": "dpkg -i /updates/webserver_v2.0.0.deb"
    },
    {
        "序号": 4,
        "执行时间": "2024-11-21 10:15:00",
        "是否允许执行": true,
        "命令": "webserver --version"
    },
    {
        "序号": 5,
        "执行时间": "2024-11-21 10:20:00",
        "是否允许执行": false,
        "命令": "rm -rf /var/www/html/temp"
    },
    {
        "序号": 6,
        "执行时间": "2024-11-21 10:25:00",
        "是否允许执行": true,
        "命令": "systemctl start webserver"
    },
    {
        "序号": 7,
        "执行时间": "2024-11-21 10:30:00",
        "是否允许执行": true,
        "命令": "systemctl status webserver"
    }
]
"""


In [4]:
result = openai_chain.run({
            "ticket_content": ticket_content.strip(),
            "executed_commands": executed_commands.strip()
        })
ollam_result = ollama_chain.run({
            "ticket_content": ticket_content.strip(),
            "executed_commands": executed_commands.strip()
        })

/tmp/ipykernel_1568848/355422183.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = openai_chain.run({


In [7]:
from IPython.display import Markdown
Markdown(result)


### 总结报告

#### 1. 执行合规性分析

**符合：**
- 所有命令的执行步骤（除第5步外）都与工单内容一致。
**不符合：**
- 第5步的命令与工单内容不一致。

#### 2. 不一致命令及原因分析

**不一致命令：**
- 工单中的第5步命令为 `systemctl start webserver`。
- 实际执行的第5步命令为 `rm -rf /var/www/html/temp`。

**原因分析：**
- 可能的原因是执行人员在执行过程中错误地输入了命令，或者误操作导致了不正确的命令执行。
- 由于 `rm -rf /var/www/html/temp` 命令可能会删除重要文件，因此需要进一步确认该命令的意图和执行的正确性。

#### 3. 不合规命令的风险说明

**不合规命令：**
- `rm -rf /var/www/html/temp`。

**风险说明：**
- 该命令可能会删除 `/var/www/html/temp` 目录下的所有文件和子目录，这可能导致数据丢失或服务中断。
- 如果该目录包含重要数据，误操作将导致不可逆的数据损失。
- 由于该命令并非工单中所列出，可能导致服务无法正常启动。

#### 4. 提供明确的审批是否通过的结论

**结论：**
- 该次升级执行存在不合规操作，不通过审批。
- 建议：
  - 重新执行正确的命令 `systemctl start webserver`。
  - 在执行升级前，确保所有重要数据已备份。
  - 严格按照工单内容操作，避免误操作。
  - 在执行关键操作前，进行充分的测试和验证，确保操作的正确性。

**最终结论：** 不通过。需要纠正不合规操作并重新执行升级流程。

---

**备注：** 请确保在执行任何关键命令前，有详细的记录和备份，以降低风险。在确认所有步骤正确无误后，方可继续执行后续步骤。

In [8]:
Markdown(ollam_result)

### 总结报告

#### 1. 执行合规性分析
- **执行合规性：** 不完全符合。

#### 2. 不一致命令及原因分析
根据原始工单和实际执行的命令进行对比，发现如下不一致：
- 工单步骤3要求的操作是安装升级包。
- 实际执行的命令中出现了“rm -rf /var/www/html/temp”这一条命令（序号5），且未在原工单中出现。

**可能的原因：**
1. 操作者误操作，删除了不必要的临时文件夹。
2. 系统管理员或运维人员对工单理解错误，导致执行了额外的步骤。

#### 3. 额外的、不合规命令操作
- **命令：** `rm -rf /var/www/html/temp`（序号5）
- **原因分析及风险说明：**
  - **原因分析：** 
    - 此命令是删除指定路径下的所有文件和子目录。
    - 在实际执行的命令中，没有明确指出该操作的目的或必要性。可能的操作者误删临时文件夹或者对工单理解有误。
  
  - **风险说明：**
    - 删除重要数据：若此“/var/www/html/temp”路径内包含重要的工作文件或其他关键文件，则可能导致业务中断或数据丢失。
    - 安全隐患：在未经过充分评估的情况下随意删除文件，可能会影响到系统的运行稳定性。

#### 4. 结论
- **审批是否通过的结论：** 不通过。原因在于实际操作中存在额外的操作步骤，并且这些操作与原工单不符，存在一定风险。
  
### 建议措施
1. 对所有运维人员进行培训，确保严格按照工单执行任务。
2. 在执行重要命令前，务必再次确认其必要性及影响范围。
3. 引入审批机制，在关键步骤的操作前后增加复核流程。

### 总结报告结束

请根据此总结报告进一步采取相应措施，以确保后续操作的合规性和安全性。